In [ ]:
#####################################################################
## Install required packages
# ! pip install hdbscan
# ! pip install xlsxwriter
# ! pip install eli5
# ! pip install shap
# ! pip install -U ipykernel
# ! pip install mglearn
# ! pip install imblearn
# ! pip install sklearn
# ! pip install delayed

In [ ]:
# 메모리에 남아있는 데이터나 변수 전부 삭제
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

In [ ]:
#####################################################################
#필요한 패키지 읽어들이기
import numpy as np
import pandas as pd
import mat73
import random
import calendar
import seaborn as sns
from scipy.stats import mstats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.python.keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization

import openpyxl
import xlsxwriter


In [ ]:
# 많은 행이나 열을 보기위한 출력 표시
pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

In [ ]:
#####################################################################
## Settings for the calculations that can be changed
########################################################################

HZ = 1  # Number of months to forecasting, months of lagged FC, must be larger than or equal to 0, forecasting_horizon
Minimum_Number_Stocks = 0.6   # Firm characteritics that are avvailable for at least x% of stocks
MicroCapLevel = 40 # The cut-off point of Micro-cap Stocks: bottom x% of KSE stocks
No_Layers = 1  # DNN
Upper_limit = 0.98       # FC upper outliers
Lower_limit = 1-Upper_limit  # FC lower outliers
Lags_Acc = 3  # the number of months for the release of acc information after acc year.

train_period = 12  # months for traning, must be larger than 1 
validation_period =3 # months for validation, must be larger than 1
multiple_STD = 1   # multiples of standard deviation for downside risk


In [ ]:
#####################################################################
# Winsorizing variables (to control outliers)
#####################################################################
def winsorize_df(df):
    return df.apply(winsorize_series, axis=0)

def winsorize_series(s):
    return mstats.winsorize(s, limits=[Lower_limit, 1-Upper_limit])    

In [ ]:
# ########################################################################
# ## Get Data  구글 드라이버에서 데이터 읽는 경우 구글드라이버를 마운트해야함
# ########################################################################
# from google.colab import drive
# drive.mount('/gdrive')


In [ ]:
########################################################################
## Get Data  
########################################################################

## =====================================================================
## I. Market Macro Data - Monthly
## ---------------------------------------------------------------------

# Macro_Variables = pd.read_excel('/gdrive/MyDrive/머신러닝과 경제예측2022/data/퀀트 시장경제데이터_월별.xlsx', sheet_name='시장데이터')   # 구글 드라이버에서 데이터 읽는 경우 
Macro_Variables = pd.read_excel("D:\\Qsync\\ACDSKK\\Lectures\\머신러닝과 경제예측2022\\data\\퀀트 시장경제데이터_월별.xlsx", sheet_name='시장데이터')
Dates = Macro_Variables['DATE'].dt.strftime('%Y%m').values
# Dates = pd.date_range(start='2000-01-31', end='2021-11-30', freq="M")
# Dates = Dates.strftime('%Y%m')
# Dates =  Dates.values

# Macro_Variables = pd.DataFrame(Macro_Variables.iloc[:,1:])   # Remove dates
# Macro_Variables.index = Dates


## ---------------------------------------------------------------------
# The Number of Months
## ---------------------------------------------------------------------

T = len(Macro_Variables)


## ---------------------------------------------------------------------
# Treatment of Market-wide Variables
## ---------------------------------------------------------------------

RET_Mkt = (Macro_Variables['KOSPI']-Macro_Variables['KOSPI'].shift(1))/Macro_Variables['KOSPI'].shift(1)*100
CS = Macro_Variables['BBB-'] - Macro_Variables['AA-']
TS = Macro_Variables['TB10Y'] - Macro_Variables['TB1Y']
SP500 = (Macro_Variables['SP500']-Macro_Variables['SP500'].shift(1))/Macro_Variables['SP500'].shift(1)*100   # S&P500 return
CN = (Macro_Variables['CN']-Macro_Variables['CN'].shift(1))/Macro_Variables['CN'].shift(1)*100   # Shanghai Index return
IPI = (Macro_Variables['IPI'].shift(1)-Macro_Variables['IPI'].shift(2))/Macro_Variables['IPI'].shift(2)*100   # Industrial Production, one month lagged
PE = Macro_Variables['PE']  # P/E ratio
CD = Macro_Variables['CD']  # CD rate
KVIX = Macro_Variables['KVIX']  # CD rate


## ---------------------------------------------------------------------
## Market Variables for the Prediction of Returns
## ---------------------------------------------------------------------
# Test Variables  - No Macro variables
Macro_Variables = pd.DataFrame(np.ones(T), columns = ['const'])
Macro_Variable_names = Macro_Variables.columns

# # Test Variables  - with Macro-variables
# Macro_Variables = pd.concat([IPI], axis=1)
# Macro_Variables.columns = ['IPI']
# Macro_Variables = pd.DataFrame(sm.add_constant(Macro_Variables))
# Macro_Variable_names = Macro_Variables.columns

## ---------------------------------------------------------------------
# Confirm Macro Variables for the prediction of returns
## ---------------------------------------------------------------------

print(Macro_Variables )
print(Macro_Variable_names )

     const
0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
5      1.0
6      1.0
7      1.0
8      1.0
9      1.0
10     1.0
11     1.0
12     1.0
13     1.0
14     1.0
15     1.0
16     1.0
17     1.0
18     1.0
19     1.0
20     1.0
21     1.0
22     1.0
23     1.0
24     1.0
25     1.0
26     1.0
27     1.0
28     1.0
29     1.0
30     1.0
31     1.0
32     1.0
33     1.0
34     1.0
35     1.0
36     1.0
37     1.0
38     1.0
39     1.0
40     1.0
41     1.0
42     1.0
43     1.0
44     1.0
45     1.0
46     1.0
47     1.0
48     1.0
49     1.0
50     1.0
51     1.0
52     1.0
53     1.0
54     1.0
55     1.0
56     1.0
57     1.0
58     1.0
59     1.0
60     1.0
61     1.0
62     1.0
63     1.0
64     1.0
65     1.0
66     1.0
67     1.0
68     1.0
69     1.0
70     1.0
71     1.0
72     1.0
73     1.0
74     1.0
75     1.0
76     1.0
77     1.0
78     1.0
79     1.0
80     1.0
81     1.0
82     1.0
83     1.0
84     1.0
85     1.0
86     1.0
87     1.0
88     1.0
89     1.0

In [ ]:
## =====================================================================
## II. Individual Stock Market Data - Monthly
## ---------------------------------------------------------------------

## ---------------------------------------------------------------------
## Reading Market Data
## ---------------------------------------------------------------------

# FC_M = pd.read_excel("/gdrive/MyDrive/머신러닝과 경제예측2022/data/퀀트 개별기업시장데이터_월별.xlsx", sheet_name = None)   # 구글 드라이버에서 데이터 읽는 경우 
FC_M = pd.read_excel("D:\\Qsync\\ACDSKK\\Lectures\\머신러닝과 경제예측2022\\data\\퀀트 개별기업시장데이터_월별.xlsx", sheet_name = None)
FC_M_Variable_Names = list(FC_M.keys())   # names of variables


## ---------------------------------------------------------------------
## Calculation of Returns
## ---------------------------------------------------------------------

ret = FC_M['ret']      # in percent
Dates_ret = ret.iloc[:,0].dt.strftime('%Y%m').values
if np.array_equal(Dates,Dates_ret):
    ret = ret.iloc[:,1:]/100    # in decimal
    ret.index = Dates
    Firm_Names = ret.columns
    ret.columns = range(0,3382)

## Returns that are used to be forecasted
RET = ret # returns of individual stocks at time t  
if HZ>1:
    for hz in range(1,HZ):
        RET = (1+RET)*(1+ret.shift(hz))-1    # shift 양수는 과거 (lag), 음수는 미래
RET.index = Dates
RET = RET*100


## ---------------------------------------------------------------------
## The Number of Stocks
## ---------------------------------------------------------------------

N = len(ret.T)


## ---------------------------------------------------------------------
## Entire Individual Stock Market Data in the form of T by N  -  remioving the first column (dates)
## ---------------------------------------------------------------------

for fc_m in FC_M_Variable_Names:
    Ind_Var = FC_M[fc_m]
    Ind_Var_Dates = Ind_Var.iloc[:,0].dt.strftime('%Y%m').values
    if not np.array_equal(Dates,Ind_Var_Dates):
        print([fc_m+" : Dates of Macro Data and Individual Stock Data Do Not Match!"])
    else:
        FC_M[fc_m] = Ind_Var.iloc[:,1:]
        FC_M[fc_m].index = Dates
        FC_M[fc_m].columns = range(0,3382)


## ---------------------------------------------------------------------
## Variables for the Prediction of Returns
## ---------------------------------------------------------------------

FC_M['trad_ind'] = pd.DataFrame(FC_M['trad_ind']/FC_M['tvolm']*10000/100)    # in percent
# FC_M['trad_ind'].index = Dates

FC_M['trad_institution'] = pd.DataFrame(FC_M['trad_institution']/FC_M['tvolm']*10000/100)    # in percent
# FC_M['trad_institution'].index = Dates

FC_M['trad_foreign'] = pd.DataFrame(FC_M['trad_foreign']/FC_M['tvolm']*10000/100)    # in percent
# FC_M['trad_foreign'].index = Dates

FC_M['tvolm'] = pd.DataFrame(FC_M['tvolm']/FC_M['m_value']/1000000*100)  # in percent
# FC_M['tvolm'].index = Dates

FC_M['short_int'] = pd.DataFrame(FC_M['short_int']/FC_M['shrout']*100)   # in percent
# FC_M['short_int'].index = Dates

# Age (years) since IPO in years
TT1 = np.trunc(FC_M['ipo_date']/10000)
TT3 = np.trunc((Dates.astype(float))/100)
TT2 = np.empty((T,N),)
for i in range(0,N):
    TT2[:,i] = TT3 - TT1.iloc[:,i]
FC_M['ipo_date'] = pd.DataFrame(np.where(TT2 <= 0, np.nan, TT2), index=Dates)    
FC_M['age'] = FC_M.pop('ipo_date')

FC_M['m_value'] = pd.DataFrame(FC_M['m_value'])
# FC_M['m_value'].index = Dates
 
FC_M['anal_rating'] = pd.DataFrame(FC_M['anal_rating'])
# FC_M['anal_rating'].index = Dates 


## ---------------------------------------------------------------------
## Saved for other calculations
## ---------------------------------------------------------------------

# nominal price
PRC = FC_M.pop('price')

TVOLM = FC_M['tvolm']  # in percent
M_VALUE = FC_M['m_value']

# total number of shares
SHROUT =pd.DataFrame(FC_M['shrout'], index=Dates)    

# Micro-cap stocks 
microcapind = pd.DataFrame(np.zeros((T,N)), index=Dates)
for t in range(0,T):
    microcapind.iloc[t,(FC_M['m_value'].iloc[t,:] < stats.scoreatpercentile(FC_M['m_value'].iloc[t,:].replace(0,np.nan).dropna(),MicroCapLevel))] = 1   # 하위 10% 보다 적은 모든 주식
    microcapind.iloc[t,(ret.iloc[t-11:t,:]==0).sum()>2] = 1     # 거래월수가 없는 달이 3달 이상이면   사용하지 않음

# ## ---------------------------------------------------------------------
# ## Remove Variables in FC_M that are not used for the prediction of returns
# ## ---------------------------------------------------------------------

for fc_m in list(FC_M):
    if fc_m in ['ret', 'mkt', 'acc_year', 'fgsc']:
        del FC_M[fc_m]
 
## ---------------------------------------------------------------------
## Saved Firm Characteristics Variables for the prediction of returns
## ---------------------------------------------------------------------

FC_M_D = {}
Items = list(FC_M.keys())
for k in Items:
    if k not in ['age', 'prc', 'exch', 'm_value']:
        FC_M_D[k+'_D'] = FC_M[k] - FC_M[k].shift(1)
FC_M.update(FC_M_D) 
del FC_M_D

## 'm_value'는 금액단위로 되어 있음
FC_M['m_value_D'] = np.log1p(FC_M['m_value']) - np.log1p(FC_M['m_value'].shift(1))
FC_M['m_value'] = np.log1p(FC_M['m_value'])


In [ ]:
## ---------------------------------------------------------------------
## Saved Firm Characteristics Variables for the prediction of returns
## ---------------------------------------------------------------------
fnames = list(FC_M.keys())
print(len(fnames))
print(fnames)
for k in fnames:
    print(FC_M[k].index[0], FC_M[k].columns[0], k)

17
['shrout', 'tvolm', 'm_value', 'short_int', 'anal_rating', 'trad_institution', 'trad_ind', 'trad_foreign', 'age', 'shrout_D', 'tvolm_D', 'short_int_D', 'anal_rating_D', 'trad_institution_D', 'trad_ind_D', 'trad_foreign_D', 'm_value_D']
200001 0 shrout
200001 0 tvolm
200001 0 m_value
200001 0 short_int
200001 0 anal_rating
200001 0 trad_institution
200001 0 trad_ind
200001 0 trad_foreign
200001 0 age
200001 0 shrout_D
200001 0 tvolm_D
200001 0 short_int_D
200001 0 anal_rating_D
200001 0 trad_institution_D
200001 0 trad_ind_D
200001 0 trad_foreign_D
200001 0 m_value_D


In [ ]:
## =====================================================================
## III. Individual Stock Accounting Data - Monthly
## ---------------------------------------------------------------------
## FC_M에 있는 market data ('tvolm', 'm_value', 'short_int', 'anal_rating', 'trad_institution', 'trad_ind', 'trad_foreign', 'age')에 다음 회계항목을 추가

## 모든 재무상태표 항목은 자산은 1로 표준화되어 있음, 자산만 금액으로 표시
# Acc_BS_Items = pd.read_excel("/gdrive/MyDrive/머신러닝과 경제예측2022/data/퀀트 S_Acc_BS.xlsx", sheet_name = None, index_col=0)       # 구글 드라이버에서 데이터 읽는 경우 
Acc_BS_Items = pd.read_excel("D:\\Qsync\\ACDSKK\\Lectures\\머신러닝과 경제예측2022\\data\\퀀트 S_Acc_BS.xlsx", sheet_name = None, index_col=0)
Acc_BS_Items_D = {}
Items = list(Acc_BS_Items.keys())
for k in Items:
    Acc_BS_Items[k] = Acc_BS_Items[k].shift(12+Lags_Acc)
    Acc_BS_Items[k].columns = range(0,3382)
    Acc_BS_Items_D[k+'_D'] = Acc_BS_Items[k].shift(12+Lags_Acc) - Acc_BS_Items[k].shift(12+12+Lags_Acc)
    Acc_BS_Items_D[k+'_D'].columns = range(0,3382)
    Acc_BS_Items_D[k+'_D'].index = Dates
FC_M.update(Acc_BS_Items) 
FC_M.update(Acc_BS_Items_D) 

## 모든 이익계산서 항목은 매출을 1로 표준화되어 있음, 매출액만 금액으로 표시
# Acc_IS_Items = pd.read_excel("/gdrive/MyDrive/머신러닝과 경제예측2022/data/퀀트 S_Acc_IS.xlsx", sheet_name = None, index_col=0)        # 구글 드라이버에서 데이터 읽는 경우 
Acc_IS_Items = pd.read_excel("D:\\Qsync\\ACDSKK\\Lectures\\머신러닝과 경제예측2022\\data\\퀀트 S_Acc_IS.xlsx", sheet_name = None, index_col=0)
Acc_IS_Items_D = {}
Items = list(Acc_IS_Items.keys())
for k in Items:
    Acc_IS_Items[k] = Acc_IS_Items[k].shift(12+Lags_Acc)
    Acc_IS_Items[k].columns = range(0,3382)
    Acc_IS_Items_D[k+'_D'] = Acc_IS_Items[k].shift(12+Lags_Acc) - Acc_IS_Items[k].shift(12+12+Lags_Acc)
    Acc_IS_Items_D[k+'_D'].columns = range(0,3382)
    Acc_IS_Items_D[k+'_D'].index = Dates
FC_M.update(Acc_IS_Items) 
FC_M.update(Acc_IS_Items_D) 

## 매출과 자산과의 비율 
FC_M['매출액_자산'] = FC_M['매출액']/FC_M['자산']
FC_M['매출액_자산_D'] = FC_M['매출액_자산'] - FC_M['매출액_자산'].shift(12)

FC_M['자산'] = np.log1p(FC_M['자산'])
FC_M['매출액'] = np.log1p(FC_M['매출액'])



<ipython-input-12-b7ce85f2ca86>:39: RuntimeWarning: invalid value encountered in log1p
  FC_M['매출액'] = np.log1p(FC_M['매출액'])


In [ ]:
## ---------------------------------------------------------------------
## Saved Firm Characteristics Variables for the prediction of returns
## ---------------------------------------------------------------------
fnames = list(FC_M.keys())
print(len(fnames))
print(fnames)
for k in fnames:
    print(FC_M[k].index[0], FC_M[k].columns[0], k)

135
['shrout', 'tvolm', 'm_value', 'short_int', 'anal_rating', 'trad_institution', 'trad_ind', 'trad_foreign', 'age', 'shrout_D', 'tvolm_D', 'short_int_D', 'anal_rating_D', 'trad_institution_D', 'trad_ind_D', 'trad_foreign_D', 'm_value_D', '자산', '기타유형자산', '단기차입금', '대출채권', '매출채권', '무형자산', '미수금', '부채', '비유동부채', '사채1', '산업재산권', '설비자산', '순운전자본', '영업고정자산', '영업권', '우선주', '유동부채', '유동성장기부채', '유동자산', '유보액', '유형자산', '이연법인세자산', '이익잉여금', '자기주식', '자본', '장기금융자산', '장기미지급금', '장기차입금', '재고자산', '전환사채', '주식발행초과금', '토지', '현금및현금성자산', '자산_D', '기타유형자산_D', '단기차입금_D', '대출채권_D', '매출채권_D', '무형자산_D', '미수금_D', '부채_D', '비유동부채_D', '사채1_D', '산업재산권_D', '설비자산_D', '순운전자본_D', '영업고정자산_D', '영업권_D', '우선주_D', '유동부채_D', '유동성장기부채_D', '유동자산_D', '유보액_D', '유형자산_D', '이연법인세자산_D', '이익잉여금_D', '자기주식_D', '자본_D', '장기금융자산_D', '장기미지급금_D', '장기차입금_D', '재고자산_D', '전환사채_D', '주식발행초과금_D', '토지_D', '현금및현금성자산_D', '매출액', '감가상각비', '관리비', '광고선전비', '당기순손익', '대손상각비(판관비)', '매출원가', '매출총이익(손실)', '무형자산상각비', '법인세비용', '법인세비용차감전순손익', '복리후생비', '부가가치', '영업외비용', '

In [ ]:
#####################################################################
# Data Treatment
#####################################################################


#for sdate in range(train_period + HZ, len(Dates)-HZ):   # 이 옵션은 처음부터 마지막 조사기간 전체를 분석 
for sdate in range(260, 261):                                   #   특정한 달만 선택하는 경우
    #sdate = train_period + HZ
    print(Dates[sdate])                                     # 계산하는 달을 보여줌

    ss_ret = pd.Series(dtype='float64')
    ss_ret_v = pd.Series(dtype='float64')
    NFCSS = pd.DataFrame()
    NFCSS_v = pd.DataFrame()    
    FFC = pd.DataFrame()  
    FFC_v = pd.DataFrame()    

    for t in range(sdate - train_period - validation_period+1, sdate+HZ):   # past train_period

        if t<=sdate- validation_period:
            print([Dates[t],"for Training"])
            s_ret = RET.iloc[t,:].dropna()
            s_ret = s_ret<s_ret.mean()-multiple_STD*s_ret.std()
            s_ret = s_ret*1    # 1 when extreme negative return, 0 otherwise

            ST = s_ret.index        
            ss_ret = pd.concat([ss_ret,s_ret])

            FC = pd.DataFrame()
            for fname in fnames:
                FC[fname] = pd.DataFrame(FC_M[fname]).iloc[t-HZ, ST] # Remove firms whose returns are not available

            # remoce unnecessary frim characteristics like return and price.
            #FC = FC.drop(['ret', 'prc'], axis=1)   
            # FCS = FC.drop(['ret', 'prc','exchcd','microcapind', 'shrout', 'sic2', 'vol'], axis=1)
            FCS = FC     # nothing to drop

            #####################################################################
            # Winsorizing and Normalising FC (to control outliers)
            WFCS = winsorize_df(FCS)
            NFCS = (WFCS-np.mean(WFCS))/np.std(WFCS)
            NFCSS = pd.concat([NFCSS,NFCS])
        

        elif t<=sdate:   
            print([Dates[t],"for Validation"])
            s_ret_v = RET.iloc[t,ST]
            s_ret_v = s_ret_v<s_ret_v.mean()-multiple_STD*s_ret_v.std()
            s_ret_v = s_ret_v*1    # 1 when extreme negative return, 0 otherwise

            ss_ret_v = pd.concat([ss_ret_v,s_ret_v])

            FFC_v = pd.DataFrame()
            for fname in fnames:        
                FFC_v[fname] = pd.DataFrame(FC_M[fname]).iloc[t-HZ, ST] # t FCs, Out-of-sample Forecasting
            # FCS_v = FFC_v.drop(['ret', 'prc','exchcd','microcapind', 'shrout', 'sic2', 'vol'], axis=1)
            FCS_v = FFC_v   # nothing to drop

            WFCS_v = winsorize_df(FCS_v)
            NFCS_v = (WFCS_v-np.mean(WFCS_v))/np.std(WFCS_v)
            NFCSS_v = pd.concat([NFCSS_v,NFCS_v])


        if t==sdate:   
            print([Dates[t+HZ],"for Testing"])
            fs_ret = RET.iloc[t+HZ,ST]    # t+1 realised return, Out-of-sample Forecasting
            fs_ret = fs_ret<fs_ret.mean()-multiple_STD*fs_ret.std()
            fs_ret = fs_ret*1  # 1 when extreme negative return, 0 otherwise

            for fname in fnames:        
                FFC[fname] = pd.DataFrame(FC_M[fname]).iloc[t, ST] # t FCs, Out-of-sample Forecasting
            # FFCS = FFC.drop(['ret', 'prc','exchcd','microcapind', 'shrout', 'sic2', 'vol'], axis=1)
            FFCS = FFC  
            WFFCS = winsorize_df(FFCS)
            NFFCSS = (WFFCS-np.mean(WFFCS))/np.std(WFFCS)


    #####################################################################
    # Selection of Firm Characteristics that are available more than the least Minimum_Number_Stocks
    SST = ss_ret.index
    Selected_FC = NFCSS.isnull().sum() <= Minimum_Number_Stocks * len(SST)
    NFCSS = NFCSS.loc[:, Selected_FC]
    NFCSS_v = NFCSS_v.loc[:, Selected_FC]    
    NFFCSS = NFFCSS.loc[:, Selected_FC]

    # Replacing NA  to 0
    NFCSS = NFCSS.fillna(0)
    NFCSS_v = NFCSS_v.fillna(0)    
    NFFCSS = NFFCSS.fillna(0)
    ss_ret = ss_ret.fillna(0)
    ss_ret_v = ss_ret_v.fillna(0)
    fs_ret = fs_ret.fillna(0)
    ST_Predict = NFFCSS.index

    NFCSS_name = NFCSS.columns
    NFCSS_v_name = NFCSS_v.columns    
    NFFCSS_name = NFFCSS.columns


    ########################################################################
    #    choose the model (Logit, Random Forest Classification, SVM)               #
    ########################################################################
    # Test Data는 알고리즘과 관계없이 일정
    X_test, y_test =  NFFCSS, fs_ret


202109
['202007', 'for Training']
['202008', 'for Training']
['202009', 'for Training']
['202010', 'for Training']
['202011', 'for Training']
['202012', 'for Training']
['202101', 'for Training']
['202102', 'for Training']
['202103', 'for Training']
['202104', 'for Training']
['202105', 'for Training']
['202106', 'for Training']
['202107', 'for Validation']
['202108', 'for Validation']
['202109', 'for Validation']
['202110', 'for Testing']


In [ ]:
    # ########################################################################
    # #            Logit Regression
    # ########################################################################
    # Estimation data
    X_train_val, y_train_val =  pd.concat([NFCSS, NFCSS_v]), pd.concat([ss_ret, ss_ret_v])    # Logit regression에는 validation set 이 필요없음
    logitmodel = LogisticRegression()
    logitmodel.fit(X_train_val, y_train_val)

    inperformance = logitmodel.score(X_train_val, y_train_val)   # 
    accuracy = logitmodel.score(X_test, y_test)   
    predicted = logitmodel.predict(X_test)

    print("================================================================")
    print("Logit Results")
    # 예측 결과와 실제 결과 Confusion Matrix출력
    print("Confusion Matrix")
    print(confusion_matrix(y_test , predicted))
    print("accuracy, precision, recall, f1-score")
    print(classification_report(y_test,predicted))
    print("Accuracy:",logitmodel.score(X_test, y_test))


Logit Results
Confusion Matrix
[[2126   11]
 [ 158    8]]
accuracy, precision, recall, f1-score
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      2137
           1       0.42      0.05      0.09       166

    accuracy                           0.93      2303
   macro avg       0.68      0.52      0.52      2303
weighted avg       0.89      0.93      0.90      2303

Accuracy: 0.9266174554928355


C:\Users\Crux\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
    ########################################################################
    #  Data,  SMOTE 을 사용하는 경우 training data oversampling
    # ########################################################################
    # training,  validation data
    X_train,X_val,y_train, y_val = NFCSS, NFCSS_v, ss_ret, ss_ret_v

    # SMOTE 을 사용하는 경우 training data oversampling
    # https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html
    from imblearn.over_sampling import SMOTE
    SMOTE = SMOTE(random_state=42)
    X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)
    print(["Total Number of the Original Training Observations: " + str(len(y_train))])
    print(["Total Number of Target in the Original Training Observations : " + str(y_train.sum()) + " (" + str(format(y_train.sum()/len(y_train)*100,".2f")) +"%)"])
    print(["Total Number of the SMOTE Training Observations: " + str(len(y_train_SMOTE))])
    print(["Total Number of Target in the SMOTE Training Observations : " + str(y_train_SMOTE.sum()) + " (" + str(format(y_train_SMOTE.sum()/len(y_train_SMOTE)*100,".2f")) +"%)"])




['Total Number of the Original Training Observations: 27102']
['Total Number of Target in the Original Training Observations : 1476 (5.45%)']
['Total Number of the SMOTE Training Observations: 51252']
['Total Number of Target in the SMOTE Training Observations : 25626 (50.00%)']


In [ ]:
    ########################################################################
    #  Random Forest Classification
    # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
    # ########################################################################
    
    RandomForest = RandomForestClassifier(n_estimators=5,  # 결정트리의 갯수, Default = 10, 무작정 트리 갯수를 늘리면 성능 좋아지는 것 대비 시간이 걸릴 수 있음 
                                            max_depth=10,    # 트리의 최대 깊이, default = None (완벽하게 클래스 값이 결정될 때 까지 분할), 깊이가 깊어지면 과적합될 수 있으므로 적절히 제어 필요
                                            min_samples_split=10,  # 노드를 분할하기 위한 최소한의 샘플 데이터수, Default = 2, 작게 설정할 수록 분할 노드가 많아져 과적합 가능성 증가 
                                            oob_score=True)   # 선택되지 않은 샘플을 사용하여 정확성 측정
    RandomForest.fit(X_train, y_train)
    predicted = RandomForest.predict(X_test)
    accuracy = accuracy_score(y_test, predicted)
    inperformance = RandomForest.oob_score_
    
    print("================================================================")
    print("Random Forest Classification Results")
    print(f'Out-of-bag score estimate: {RandomForest.oob_score_:.3}')
    print(f'Mean accuracy score: {accuracy:.3}')
    # 예측 결과와 실제 결과 Confusion Matrix출력
    print("Confusion Matrix")
    print(confusion_matrix(y_test , predicted))
    print("accuracy, precision, recall, f1-score")   
    print(classification_report(y_test,predicted))


    # SMOTE 을 사용하는 경우
    RandomForest.fit(X_train_SMOTE, y_train_SMOTE)
    predicted = RandomForest.predict(X_test)
    accuracy = accuracy_score(y_test, predicted)
    inperformance = RandomForest.oob_score_
    
    print("-----------------------------------------------------------------")
    print("Random Forest Classification Results with SMOTE")    
    print(f'Out-of-bag score estimate with SMOTE: {RandomForest.oob_score_:.3}')
    print(f'Mean accuracy score  with SMOTE: {accuracy:.3}')
    # 예측 결과와 실제 결과 Confusion Matrix출력
    print("Confusion Matrix")
    print(confusion_matrix(y_test , predicted))
    print("accuracy, precision, recall, f1-score")      
    print(classification_report(y_test,predicted))





C:\Users\Crux\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Random Forest Classification Results
Out-of-bag score estimate: 0.939
Mean accuracy score: 0.924
Confusion Matrix
[[2127   10]
 [ 164    2]]
accuracy, precision, recall, f1-score
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      2137
           1       0.17      0.01      0.02       166

    accuracy                           0.92      2303
   macro avg       0.55      0.50      0.49      2303
weighted avg       0.87      0.92      0.89      2303

-----------------------------------------------------------------
Random Forest Classification Results with SMOTE
Out-of-bag score estimate with SMOTE: 0.786
Mean accuracy score  with SMOTE: 0.814
Confusion Matrix
[[1788  349]
 [  80   86]]
accuracy, precision, recall, f1-score
              precision    recall  f1-score   support

           0       0.96      0.84      0.89      2137
           1       0.20      0.52      0.29       166

    accuracy                           0.81      23

C:\Users\Crux\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [ ]:
    # ########################################################################
    # #           Support Vector Machine
    # https://scikit-learn.org/stable/modules/svm.html
    # ########################################################################

    SVM_Classifier = svm.SVC(kernel="rbf", C = 2, gamma = 0.5)      # kernel='linear' 'rbf' (Radial Bias Function),‘sigmoid’, ‘precomputed’
                                                                    # gamma: 결정 경계를 얼마나 유연하게 그을 것인지, C: 오류허용정도 (크면 오류허용이 적어짐)
    SVM_Classifier.fit(X_train, y_train)
    predicted = SVM_Classifier.predict(X_test)
    inperformance = SVM_Classifier.score(X_train, y_train)
    accuracy = SVM_Classifier.score(X_test, y_test)

    print("================================================================")
    print("Support Vector Machine Results")  
    print("Accuracy:",SVM_Classifier.score(X_test, y_test))
    # 예측 결과와 실제 결과 Confusion Matrix출력
    print("Confusion Matrix")
    print(confusion_matrix(y_test , predicted))
    print("accuracy, precision, recall, f1-score")      
    print(classification_report(y_test,predicted))


    # SMOTE 을 사용하는 경우
    SVM_Classifier.fit(X_train_SMOTE, y_train_SMOTE)
    predicted = SVM_Classifier.predict(X_test)
    inperformance = SVM_Classifier.score(X_train_SMOTE, y_train_SMOTE)
    accuracy = SVM_Classifier.score(X_test, y_test)

    print("----------------------------------------------------------------")
    print("Support Vector Machine Results with SMOTE")  
    print("Accuracy  with SMOTE:",SVM_Classifier.score(X_test, y_test))
    # 예측 결과와 실제 결과 Confusion Matrix출력
    print("Confusion Matrix")
    print(confusion_matrix(y_test , predicted))
    print("accuracy, precision, recall, f1-score")      
    print(classification_report(y_test,predicted))





Support Vector Machine Results
Accuracy: 0.9279201042118975
Confusion Matrix
[[2137    0]
 [ 166    0]]
accuracy, precision, recall, f1-score
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      2137
           1       0.00      0.00      0.00       166

    accuracy                           0.93      2303
   macro avg       0.46      0.50      0.48      2303
weighted avg       0.86      0.93      0.89      2303



C:\Users\Crux\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Crux\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Crux\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


----------------------------------------------------------------
Support Vector Machine Results with SMOTE
Accuracy  with SMOTE: 0.9279201042118975
Confusion Matrix
[[2137    0]
 [ 166    0]]
accuracy, precision, recall, f1-score
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      2137
           1       0.00      0.00      0.00       166

    accuracy                           0.93      2303
   macro avg       0.46      0.50      0.48      2303
weighted avg       0.86      0.93      0.89      2303



C:\Users\Crux\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Crux\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Crux\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
